In [2]:
import requests
import json
import os
from dotenv import load_dotenv

# # Load environment variables from .env file
load_dotenv()

APIGEE_X_CLIENT_ID = os.getenv("APIGEE_X_KEY")
APIGEE_X_SECRET_ID = os.getenv("APIGEE_X_SECRET")

print(f"Apigee X Client ID: {APIGEE_X_CLIENT_ID}")
print(f"Apigee X Secret ID Type: {type(APIGEE_X_SECRET_ID)}")

Apigee X Client ID: ryKtQImeBMm8xGNdWUr9TdrsFquIsXpOIHzxaDLi1rNcTQAp
Apigee X Secret ID Type: <class 'str'>


In [7]:
def get_apigee_x_token():
    """Acquires an OAuth token for Apigee X."""
    base_url = "https://dev.mcc.apix.mayo.edu/" # Note: Trailing slash is often part of base URL for Apigee X
    oauth_url = base_url + 'oauth/token' # Note: No leading slash here if base_url ends with one

    payload = f'grant_type=client_credentials&client_id={APIGEE_X_CLIENT_ID}&client_secret={APIGEE_X_SECRET_ID}'
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded'
    }

    print(f"Requesting Apigee X token from: {oauth_url}")
    response = requests.post(oauth_url, headers=headers, data=payload)
    response.raise_for_status() # Raise an exception for HTTP errors

    apigee_token = response.json().get('access_token')
    print(f'Apigee X Bearer Token acquired: {apigee_token}')
    return base_url, apigee_token

In [8]:
# Common API Parameters

# --- Common Chat/Completion Parameters ---
# Define a default context and prompt for chat/completion models.
# These can be overridden in specific test cells if needed.
CHAT_COMPLETION_CONTEXT = "You are a knowledgeable and trustworthy medical assistant. Answer queries with accurate, concise, and medically verified information."
CHAT_COMPLETION_PROMPT = "What is methotrexate?"
CHAT_COMPLETION_MAX_TOKENS = 4000  # Limit to ensure concise responses
CHAT_COMPLETION_TEMPERATURE = 0.0  # Low temperature for factual, deterministic responses
CHAT_COMPLETION_TOP_P = 1.0  # Consider the full probability distribution
CHAT_COMPLETION_PRESENCE_PENALTY = 0.0  # No penalty for introducing new topics
CHAT_COMPLETION_FREQUENCY_PENALTY = 0.0  # No penalty for repeating information


# --- Common Embeddings Parameter ---
# Define a default text input for embeddings models.
EMBEDDING_TEXT_INPUT = "What is the capital of France?"


# --- General API Request Helper Function ---
def make_api_request(full_url, token, payload, is_embeddings=False):
    """
    Makes an API request to Azure OpenAI and parses the response.

    Args:
        full_url (str): The complete URL for the API endpoint.
        token (str): The bearer token for authorization.
        payload (dict): The JSON payload for the request.
        is_embeddings (bool): True if testing an embeddings model, False for chat/completions.

    Returns:
        dict: The parsed JSON response, or None if an error occurred.
    """
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    print(f"\nMaking API call to: {full_url}")
    print(f"Payload (first 100 chars): {json.dumps(payload)[:100]}...")

    try:
        response = requests.post(full_url, headers=headers, json=payload)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

        json_object = response.json()

        # Print debugging information based on API type
        print(f"API Response Status: {response.status_code}")
        print(f"Tokens used (prompt): {json_object.get('usage', {}).get('prompt_tokens', 'N/A')}")
        print(f"Tokens used (completion): {json_object.get('usage', {}).get('completion_tokens', 'N/A')}")
        print(f"Tokens used (total): {json_object.get('usage', {}).get('total_tokens', 'N/A')}")

        if is_embeddings:
            embeddings_data = json_object.get('data', [])
            if embeddings_data:
                first_embedding = embeddings_data[0].get('embedding', [])
                print(f"Embedding received (first 5 values): {first_embedding[:5]}...")
                print(f"Total embedding dimension: {len(first_embedding)}")
            else:
                print("No embedding data found in the response 'data' field.")
        else: # Chat/Completions
            content = json_object.get('choices', [{}])[0].get('message', {}).get('content', "No content available")
            print(f"Answer: {content[:200]}...") # Print first 200 chars of answer

        return json_object

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {response.text}")
    except json.JSONDecodeError as json_err:
        print(f"JSONDecodeError: Could not decode JSON response: {json_err} - {response.text}")
    except Exception as err:
        print(f"An unexpected error occurred: {err}")

    return None

In [10]:
# Test Apigee X - Chat/Completions

print("\n--- Testing Apigee X: Chat/Completions ---")

# 1. Acquire token for Apigee X
try:
    apix_url, apix_token = get_apigee_x_token()
except Exception as e:
    print(f"Failed to get Apigee X token: {e}")
    apix_token = None # Ensure token is None if acquisition fails

if apix_token:
    # 2. Configure model and API version
    # chat_engine = 'gpt-4o'
    # chat_engine = 'gpt-4o-mini'
    # chat_engine = 'o1'
    # chat_engine = 'o3-mini'
    chat_engine = 'o4-mini'
    # chat_engine = 'gpt-4.1'
    # chat_engine = 'gpt-4.1-mini'
    # chat_engine = 'gpt-4.1-nano'
    chat_api_version = '2025-04-01-preview'

    # 3. Prepare payload for Chat/Completions
    chat_payload = {
        "messages": [
            {"role": "system", "content": CHAT_COMPLETION_CONTEXT},
            {"role": "user", "content": CHAT_COMPLETION_PROMPT}
        ],
        "top_p": CHAT_COMPLETION_TOP_P,
        "presence_penalty": CHAT_COMPLETION_PRESENCE_PENALTY,
        "frequency_penalty": CHAT_COMPLETION_FREQUENCY_PENALTY,
        # "max_tokens": CHAT_COMPLETION_MAX_TOKENS, # Uncomment if you want to include max_tokens
        # "temperature": CHAT_COMPLETION_TEMPERATURE, # Uncomment if you want to include temperature
    }

    # 4. Construct the full API URL for Apigee X Chat
    # Note the path: /llm-azure-openai/openai/... for Apigee X
    api_openai_url = apix_url + f"llm-azure-openai/openai/deployments/{chat_engine}/chat/completions?api-version={chat_api_version}"

    # 5. Make the API request
    print(f"\nDeployment Name: {chat_engine}")
    print(f"Question: {CHAT_COMPLETION_PROMPT}")
    make_api_request(api_openai_url, apix_token, chat_payload, is_embeddings=False)
else:
    print("Skipping Apigee X Chat/Completions test due to missing token.")


--- Testing Apigee X: Chat/Completions ---
Requesting Apigee X token from: https://dev.mcc.apix.mayo.edu/oauth/token
Apigee X Bearer Token acquired: obXQVeV7DHC7LueYGUIvlsPoYTnq

Deployment Name: o4-mini
Question: What is methotrexate?

Making API call to: https://dev.mcc.apix.mayo.edu/llm-azure-openai/openai/deployments/o4-mini/chat/completions?api-version=2025-04-01-preview
Payload (first 100 chars): {"messages": [{"role": "system", "content": "You are a knowledgeable and trustworthy medical assista...
API Response Status: 200
Tokens used (prompt): 38
Tokens used (completion): 705
Tokens used (total): 743
Answer: Methotrexate is a disease-modifying antirheumatic drug (DMARD) and antineoplastic agent that works as a folate antagonist. By inhibiting the enzyme dihydrofolate reductase it blocks the formation of t...


In [ ]:
https://dev.mcc.apix.mayo.edu/llm-azure-openai/openai/deployments/o4-mini/chat/completions?api-version=2025-04-01-preview